## Google Drive

In [ ]:
""" Evaluate.ipynb """

# Colab Drive Mount: 결과물을 내 드라이브에 저장하거나 불러오기 위해 필요
from google.colab import drive
drive.mount('/content/drive')

In [2]:
# Google Drive에서 데이터셋 불러오기
!cp /content/drive/MyDrive/img_align_celeba.zip /content/ # 데이터 저장 위치를 작성

!unzip -q /content/drive/MyDrive/img_align_celeba.zip -d /content/data/ # 데이터 저장 위치 작성

data_dir = '/content/data/img_align_celeba'

## Import / Cuda

In [ ]:
# Install required libraries
!pip install lpips
!pip install pytorch-fid

In [4]:
# import
import os
import random
import numpy as np
import torch

from torch.utils.data import Dataset, DataLoader, Subset
from torchvision import transforms
from torchvision.datasets import ImageFolder
from tqdm import tqdm
from PIL import Image

from pytorch_fid import fid_score
import lpips

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Dataset

In [6]:
# Custom Dataset
class SingleImageDataset(Dataset):
  def __init__(self, folder, transform=None):
    self.paths = sorted([os.path.join(folder, f) for f in os.listdir(folder) if f.endswith(('.png', '.jpg'))])
    self.transform = transform

  def __len__(self):
    return len(self.paths)

  def __getitem__(self, idx):
    img = Image.open(self.paths[idx]).convert('RGB')
    if self.transform:
      img = self.transform(img)
    return img, self.paths[idx]

In [8]:
gen_folder = '/content/drive/MyDrive/ProGAN/Final_images' # 최종 생성 이미지
real_folder = '/content/data/img_align_celeba'
sampled_folder = '/content/data/real_celebA_sampled'
os.makedirs(sampled_folder, exist_ok=True)

# 학습에 사용하지 않은 인덱스에서 50장 추출
unused_idx_path = '/content/drive/MyDrive/ProGAN/unused_indices_for_eval.npy'
unused_indices = np.load(unused_idx_path)
random.seed(42)
sample_indices = random.sample(list(unused_indices), 50)

# 이미지 복사
real_imgs = sorted([f for f in os.listdir(real_folder) if f.endswith('.jpg')])
for i, idx in enumerate(sample_indices):
  fname = real_imgs[idx]
  src = os.path.join(real_folder, fname)
  dst = os.path.join(sampled_folder, f"real_{i:02d}.jpg")
  os.system(f'cp "{src}" "{dst}"')

## Lpips metric / Dataloader

In [ ]:
# Lpips metric
lpips_fn = lpips.LPIPS(net='alex').to(device)
lpips_fn.eval()

lpips_tf = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

In [10]:
# 데이터셋 준비
gen_ds = SingleImageDataset(gen_folder, transform=lpips_tf)
real_ds = SingleImageDataset(sampled_folder, transform=lpips_tf)

N = min(len(gen_ds), len(real_ds))
gen_ds = Subset(gen_ds, list(range(N)))
real_ds = Subset(real_ds, list(range(N)))

gen_loader = DataLoader(gen_ds, batch_size=1, shuffle=False)
real_loader = DataLoader(real_ds, batch_size=1, shuffle=False)

## LPIPS 평가

In [ ]:
# LPIPS 평가
total_lpips = []
with torch.no_grad():
  for (g_img, _), (r_img, _) in zip(gen_loader, real_loader):
    g_img = g_img.to(device)
    r_img = r_img.to(device)
    dist = lpips_fn(g_img, r_img)
    total_lpips.append(dist.item())

lpips_scores = np.array(total_lpips)

print(f"Evaluated {N} pairs")
print(f"Mean LPIPS: {lpips_scores.mean():.4f}")
print(f"Median LPIPS: {np.median(lpips_scores):.4f}")
print(f"Std LPIPS: {lpips_scores.std():.4f}")

## FID 평가

In [ ]:
# FID 평가
fid_value = fid_score.calculate_fid_given_paths(
    [sampled_folder, gen_folder],
    batch_size=50,
    device=device,
    dims=2048
)

print(f"FID Score: {fid_value:.4f}")